In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
from pathlib import Path
import concurrent.futures

# Read data

In [2]:
# Read dictionary with municipalities
# Com a pathlib nao precisamos ficar modificando o caminho dos arquivos, basta deixar todos no mesmo diretorio
# muni = pd.read_csv('DTB_BRASIL_MUNICIPIO.csv',sep = ';')
muni = pd.read_csv('/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/data/DTB_BRASIL_MUNICIPIO.csv',sep=';')    
    

In [3]:
muni = muni[['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município']]

In [4]:
# Read the Adjacent matrix

link = '/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/code/'
matrix = {}
for month in range(1, 13):
    matrix[month] = pd.read_parquet(f'{link}adjacency_matrix_complete_{month}.parquet', engine='pyarrow')

In [5]:
df_matrix = {}
for month in range(1, 13):
    df_matrix = matrix[month].astype('uint16')
    df_matrix.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 1100015 to 5300108
Columns: 5570 entries, 1100015 to 5300108
dtypes: uint16(5570)
memory usage: 59.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 1100015 to 5300108
Columns: 5570 entries, 1100015 to 5300108
dtypes: uint16(5570)
memory usage: 59.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 1100015 to 5300108
Columns: 5570 entries, 1100015 to 5300108
dtypes: uint16(5570)
memory usage: 59.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 1100015 to 5300108
Columns: 5570 entries, 1100015 to 5300108
dtypes: uint16(5570)
memory usage: 59.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 1100015 to 5300108
Columns: 5570 entries, 1100015 to 5300108
dtypes: uint16(5570)
memory usage: 59.2 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 1100015 to 5300108
Columns: 5570 entries, 1100015 to 5300108
dtypes: uint16(5570)
memory usage: 

In [6]:
# hubs = pd.read_csv('lists_of_hubs.csv')
hubs = pd.read_csv('/Users/andreza/reproducibility_transportation_hubs-early_warning_surveillance_systems/data/lists_of_hubs.csv')

# Functions

In [10]:
#change matrix here
link_muni_vertice = pd.DataFrame(matrix[4].columns, columns=['muni'])

In [11]:
def get_mname(n):
    
    m = link_muni_vertice.iloc[n]['muni']
    set_muni = muni[muni['Código Município Completo'] == m].reset_index()
    return [set_muni.iloc[0]['Nome_Município'],set_muni.iloc[0]['Nome_UF'],m]
   
#def get_mnumber(name):
#    muni[muni['Nome_Município'] == name]
#    
#    co_mu = muni[muni['Nome_Município'] == name].reset_index()['Código Município Completo'][0]
#    muni_number = link_muni_vertice[link_muni_vertice['muni'] == co_mu]['muni'].index.tolist()[0]
#    return [muni_number, co_mu]

def get_mnumber(name,uf):
    
    set_df = muni[(muni['Nome_Município'] == name) & (muni['Nome_UF'] == uf)].reset_index()
    
    co_mu = set_df['Código Município Completo'][0]
    
    muni_number = link_muni_vertice[link_muni_vertice['muni'] == co_mu]['muni'].index.tolist()[0]
    
    return [muni_number, co_mu]

In [12]:
def get_mnumber_of_set(df_col):
    muni_number = []

    for i in range(0,len(df_col)):
    
        uf = df_col['Nome_UF'].iloc[i]
    
        city = df_col['Nome_Município'].iloc[i]
    
        muni_number.append(get_mnumber(city,uf)[0])
    
    return muni_number

In [13]:
# Python program for implementation
# of Ford Fulkerson algorithm
from collections import defaultdict

# This class represents a directed graph
# using adjacency matrix representation
        
class Graph:

    def __init__(self, graph):
        self.graph = graph # residual graph
        self. ROW = len(graph)
        # self.COL = len(gr[0])


    '''Returns true if there is a path from source 's' to sink 't' in
    residual graph. Also fills parent[] to store the path '''


    def BFS(self, s, t, parent):
        visited = [False] * self.ROW
        queue = []
        queue.append(s)
        visited[s] = True

        while queue:
            u = queue.pop(0)
            for ind, val in enumerate(self.graph[u]):
                if visited[ind] == False and val > 0:
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u
                    if ind == t:
                        return True
        return False

    def FordFulkerson(self, source, sink):
        parent = [-1] * self.ROW
        max_flow = 0
        paths = []  # List to store the paths
        value_path = []

        while self.BFS(source, sink, parent):
            path_flow = float("inf")
            s = sink
            while s != source:
                path_flow = min(path_flow, self.graph[parent[s]][s])
                s = parent[s]

            max_flow += path_flow

            # Store the current path
            current_path = []
            v = sink
            while v != source:
                u = parent[v]
                current_path.append((u, v))
                self.graph[u][v] -= path_flow
                self.graph[v][u] += path_flow
                v = parent[v]
            paths.append(current_path)
            value_path.append(max_flow)
            
            if len(current_path) > 3:
                break

        df_paths = pd.DataFrame(paths).transpose()
        return max_flow, df_paths,value_path

# Define the list of municipalities as the origen and destination to run the FF method

## Destination

In [14]:
selec_hubs = hubs[hubs['hub_inter'] == 1]

In [15]:
selec_hubs.head()

Unnamed: 0    Nome_UF  UF2  co_ibge       Nome_Município  ind_proxi  \
1           1       Pará   15  1502954  Eldorado do Carajás   0.350491   
2           2  Tocantins   17  1702554       Augustinópolis   0.333847   
4           4   Maranhão   21  2108207            Pedreiras   0.346127   
7           7    Paraíba   25  2517001            Umbuzeiro   0.314568   
9           9      Bahia   29  2926301   Riachão do Jacuípe   0.376729   

   ind_intermed  ind_proxi_per  ind_intermed_per   grau  densidade_2022  \
1      0.000296      35.049146          0.029642  104.0            9.53   
2      0.000319      33.384663          0.031884   37.0           44.97   
4      0.000123      34.612683          0.012277   36.0          141.38   
7      0.000081      31.456761          0.008081   15.0           49.17   
9      0.000247      37.672899          0.024668   49.0           28.90   

   populacao_2022  ones  hub_ind_proxi  hub_ind_intermed  hub_density  \
1           28192     1            0.0               1.0          0.0   
2           17484     1            0.0               1.0          0.0   
4           37050     1            0.0               1.0          1.0   
7            9124     1            0.0               1.0          0.0   
9           33386     1            1.0               1.0          0.0   

   hub_inter  
1        1.0  
2        1.0  
4        1.0  
7        1.0  
9        1.0

In [16]:
selec_hubs = selec_hubs.assign(muni_number = get_mnumber_of_set(selec_hubs))

In [17]:
# selec_hubs = selec_hubs[selec_hubs.Nome_UF == 'Amazonas']

## Origin

In [18]:
# UF = 'Acre'
# ac_muni = muni[muni.Nome_UF == UF]
# path_folder = Path(f'FF_path_results_{UF}')
# path_folder.mkdir(exist_ok=True)

#choosing a municipality
ac_muni = muni[muni.Nome_Município == 'Manaus']

In [19]:
ac_muni = ac_muni.assign(muni_number = get_mnumber_of_set(ac_muni))

In [20]:
ac_muni.head(20)

UF   Nome_UF  Mesorregião Geográfica   Nome_Mesorregião  \
111  13  Amazonas                       3  Centro Amazonense   

     Microrregião Geográfica Nome_Microrregião  Município  \
111                        7            Manaus       2603   

     Código Município Completo Nome_Município  muni_number  
111                    1302603         Manaus          111

# Run method for the list of municipalities selected

In [21]:
lst1 = list(ac_muni['muni_number']) #[52,53,54,55,56,58,59,60,61,62,63,64,65,66,67,68,69,71,72,73]
lst2 = list(selec_hubs['muni_number'])#[57,67,70, 108,119,111,102,91]#list(range(1,5570,1))

print(len(lst1), ',', len(lst2), ',', len(lst1)*len(lst2))

1 , 1391 , 1391


In [22]:
#choosing the number of MONTH
# Recife [1/4],  Manaus [7/4], Salvador[1/5],SP [1/ 6], Rio [12/6]

df_matrix_np = matrix[4].to_numpy()


In [23]:
# Define a function to process each value in parallel
def process_value(value):
    n, m = value

    #matrix = pd.read_parquet(link0, engine='pyarrow')
    #df_np = matrix.to_numpy()
    
    #com o df em .copy() não precisamos ler/carregar o arquivo varias vezes
    g = Graph(df_matrix_np.copy())

    max_flow, df_paths, value_path = g.FordFulkerson(n, m)
    
    # evitar a escrita de arquivos vazios, o que tbm reduz o tempo de execução
    
    if len(value_path)==0:
        return
    
    #create output data frame
   

    lists = [df_paths[col].tolist() for col in df_paths]
    
    path_value = pd.DataFrame(value_path, columns=['value_cum'])
    path_value = path_value.assign(value=path_value['value_cum'] - path_value['value_cum'].shift())
    path_value['value'] = path_value['value'].fillna(0)
    path_value['ori_muni_name'] = get_mname(n)[0]
    path_value['ori_uf_name'] = get_mname(n)[1]
    path_value['ori_co_ibge'] = get_mname(n)[2]
    path_value['des_muni_name'] = get_mname(m)[0]
    path_value['des_uf_name'] = get_mname(m)[1]
    path_value['des_co_ibge'] = get_mname(m)[2]
    path_value = path_value.assign(paths=lists)
    
    
    #save_output
    
    table2 = pa.Table.from_pandas(path_value)

    string2 = str(get_mname(n)[2]) + '_' + str(get_mname(m)[2]) + '_' + 'path_value'

    path_to_save2 = '/Users/andreza/Downloads/FF_path_results_Manaus_4/{}.parquet'.format(string2)

    pq.write_table(table2, path_to_save2)
    


In [24]:
%%time
# Create a list of values to process
list_orig_dest = [(x, y) for x in lst1 for y in lst2]

CPU times: user 191 µs, sys: 1 µs, total: 192 µs
Wall time: 200 µs


In [25]:
%%time

# Create a ThreadPoolExecutor to run the processing function in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(process_value, list_orig_dest)

    

CPU times: user 21h 34min 13s, sys: 16min 16s, total: 21h 50min 29s
Wall time: 21h 35min 43s


In [23]:
#em andamento
# def create_output_dataframe(path_value, df_paths, n, m):
#     lists = [df_paths[col].tolist() for col in df_paths]
    
#     path_value = pd.DataFrame(value_path, columns=['value_cum'])
#     path_value = path_value.assign(value=path_value['value_cum'] - path_value['value_cum'].shift())
#     path_value['value'] = path_value['value'].fillna(0)
#     path_value['ori_muni_name'] = get_mname(n)[0]
#     path_value['ori_uf_name'] = get_mname(n)[1]
#     path_value['ori_co_ibge'] = get_mname(n)[2]
#     path_value['des_muni_name'] = get_mname(m)[0]
#     path_value['des_uf_name'] = get_mname(m)[1]
#     path_value['des_co_ibge'] = get_mname(m)[2]
#     path_value = path_value.assign(paths=lists)
    
#     return path_value
    
    
# def save_output(n, m, path_value):
    
#     table2 = pa.Table.from_pandas(path_value)

#     string2 = str(get_mname(n)[2]) + '_' + str(get_mname(m)[2]) + '_' + 'path_value'

#     #path_to_save2 = path_folder/f'{string2}.parquet'
#     #path_to_save2 = path_folder/'{}.parquet'.format(string2)
#     path_to_save2 = '/Users/andreza/Downloads/FF_path_results_Acre/{}.parquet'.format(string2)

#     pq.write_table(table2, path_to_save2)
    

# # Define a function to process each value in parallel
# def process_value(value):
#     n, m = value

#     #matrix = pd.read_parquet(link0, engine='pyarrow')
#     #df_np = matrix.to_numpy()
    
#     #com o df em .copy() não precisamos ler/carregar o arquivo varias vezes
#     g = Graph(df_np.copy())

#     max_flow, df_paths, value_path = g.FordFulkerson(n, m)
#     #podemos utilizar esse trecho se quisermos evitar a escrita de arquivos vazios
#     if len(value_path)==0:
#         return
    
#     #create output data frame
#     df_values = create_output_dataframe(value_path, df_paths, n, m)
#     save_output(n, m, path_value)

In [24]:
#original
# for value in [(x, y) for x in lst1 for y in lst2]:#zip(lst1,lst2):
#     n,m = value
    
#     #Part 1 - carregando os dados da matriz
#     #start_time = timeit.default_timer()
    
#     matrix = pd.read_parquet(link0, engine='pyarrow')
#     df_np = matrix.to_numpy()
    
#     #elapsed = timeit.default_timer() - start_time
#     #print("Part 1: " + str(elapsed))    

#     #Part 2 - gerando o grafo
#     #start_time = timeit.default_timer()
#     graph = df_np
#     g = Graph(graph)
#     #elapsed = timeit.default_timer() - start_time
#     #print("Part 2: " + str(elapsed))
    
#     #Part 3 chamada FF
#     #start_time = timeit.default_timer()
#     max_flow, df_paths, value_path = g.FordFulkerson(n, m)
    
#     #elapsed = timeit.default_timer() - start_time
#     #print("Part 3: " + str(elapsed))
    
    
#     #Part 4
#     #start_time = timeit.default_timer()
#     # transform df int a variable of lists
#     lists = [df_paths[col].tolist() for col in df_paths]

#     #elapsed = timeit.default_timer() - start_time
#     #print("Part 4: " + str(elapsed))
    
#     #Part 5 - criando output
#     #start_time = timeit.default_timer()
    
#     #format value_path
#     path_value = pd.DataFrame(value_path, columns=['value_cum'])
#     path_value = path_value.assign(value = path_value['value_cum']-path_value['value_cum'].shift())
#     path_value['value'] = path_value['value'].fillna(0)
#     #path_value['value'].iloc[0] = path_value['value_cum'].iloc[0]
#     path_value['ori_muni_name'] = get_mname(n)[0]
#     path_value['ori_uf_name'] = get_mname(n)[1]
#     path_value['ori_co_ibge'] = get_mname(n)[2]
#     path_value['des_muni_name'] = get_mname(m)[0]
#     path_value['des_uf_name'] = get_mname(m)[1]
#     path_value['des_co_ibge'] = get_mname(m)[2]
#     #add lists to path_value
#     path_value = path_value.assign(paths = lists)
    
#     #path_value.value[0] = path_value.value_cum[0]
    
#     # Save df_paths
#     #table1 = pa.Table.from_pandas(df_paths)

#     #string1 = str(get_mname(n)[2]) + '_' + str(get_mname(m)[2]) + '_' + 'path'

#     #path_to_save1 = '../FF_path_results_Acre/{}.parquet'.format(string1)

#     #pq.write_table(table1, path_to_save1)
    
#     #save path_value
    
#     table2 = pa.Table.from_pandas(path_value)

#     string2 = str(get_mname(n)[2]) + '_' + str(get_mname(m)[2]) + '_' + 'path_value'

#     path_to_save2 = '/Users/andreza/Downloads/FF_path_results_RJ/{}.parquet'.format(string2)

#     pq.write_table(table2, path_to_save2)
    
#     #elapsed = timeit.default_timer() - start_time
#     #print("Part 5: " + str(elapsed))